In [11]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/shl-hiring-assessment/Dataset/sample_submission.csv
/kaggle/input/shl-hiring-assessment/Dataset/train.csv
/kaggle/input/shl-hiring-assessment/Dataset/test.csv
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_885.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_1142.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_1006.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_817.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_765.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_508.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_257.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_330.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_72.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_328.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/audio_858.wav
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/

In [12]:
import pandas as pd
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


np.random.seed(42)
tf.random.set_seed(42)

In [13]:
print("Available datasets in /kaggle/input/:")
!ls /kaggle/input/

TRAIN_CSV = '/kaggle/input/shl-hiring-assessment/Dataset/train.csv'
TEST_CSV = '/kaggle/input/shl-hiring-assessment/Dataset/test.csv'
TRAIN_AUDIO_DIR = '/kaggle/input/shl-hiring-assessment/Dataset/audios/train/'
TEST_AUDIO_DIR = '/kaggle/input/shl-hiring-assessment/Dataset/audios/test/'
SAMPLE_SUBMISSION = '/kaggle/input/shl-hiring-assessment/Dataset/sample_submission.csv'
print("Checking file paths:")
for path in [TRAIN_CSV, TEST_CSV, SAMPLE_SUBMISSION]:
    print(f"{path}: {'Exists' if os.path.exists(path) else 'Does NOT exist'}")
print("Checking directories:")
for dir_path in [TRAIN_AUDIO_DIR, TEST_AUDIO_DIR]:
    print(f"{dir_path}: {'Exists' if os.path.exists(dir_path) else 'Does NOT exist'}, contains {len(os.listdir(dir_path)) if os.path.exists(dir_path) else 0} files")

try:
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    sample_submission = pd.read_csv(SAMPLE_SUBMISSION)
    print("CSV files loaded successfully")
    print(f"Training samples: {len(train_df)}, Test samples: {len(test_df)}")
    print("train_df columns:", train_df.columns.tolist())
    print("test_df columns:", test_df.columns.tolist())
except Exception as e:
    print(f"Error loading CSV files: {e}")
    raise





Available datasets in /kaggle/input/:
shl-data  shl-hiring-assessment
Checking file paths:
/kaggle/input/shl-hiring-assessment/Dataset/train.csv: Exists
/kaggle/input/shl-hiring-assessment/Dataset/test.csv: Exists
/kaggle/input/shl-hiring-assessment/Dataset/sample_submission.csv: Exists
Checking directories:
/kaggle/input/shl-hiring-assessment/Dataset/audios/train/: Exists, contains 444 files
/kaggle/input/shl-hiring-assessment/Dataset/audios/test/: Exists, contains 204 files
CSV files loaded successfully
Training samples: 444, Test samples: 204
train_df columns: ['filename', 'label']
test_df columns: ['filename']


In [14]:
def extract_features(audio_path, max_length=100):
   
    try:
        audio, sr = librosa.load(audio_path, sr=22050)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        if mfccs.shape[1] > max_length:
            mfccs = mfccs[:, :max_length]
        else:
            mfccs = np.pad(mfccs, ((0, 0), (0, max_length - mfccs.shape[1])), mode='constant')
        return mfccs.flatten()
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None

In [ ]:
import os
import librosa
import numpy as np



X_train = []
y_train = []
for idx, row in train_df.iterrows():
    audio_path = os.path.join(TRAIN_AUDIO_DIR, row['filename'])
    features = extract_features(audio_path)
    if features is not None:
        X_train.append(features)
        y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)
print(f"Processed {len(X_train)} training samples")


X_test = []
test_filenames = []
for idx, row in test_df.iterrows():
    audio_path = os.path.join(TEST_AUDIO_DIR, row['filename'])
    features = extract_features(audio_path)
    if features is not None:
        X_test.append(features)
        test_filenames.append(row['filename'])

X_test = np.array(X_test)
print(f"Processed {len(X_test)} test samples")

Processed 444 training samples


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split  

from sklearn.metrics import mean_squared_error 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)
print("Data preprocessing completed")

In [ ]:

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

def build_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_model(X_train_scaled.shape[1])
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

history = model.fit(
    X_train_split, y_train_split,
    validation_data=(X_val_split, y_val_split),
    epochs=30,
    batch_size=16,
    callbacks=[early_stopping],
    verbose=1
)
print("Model training completed")

In [ ]:
# Evaluate the model on training data
train_predictions = model.predict(X_train_scaled, verbose=0)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
print(f"Training RMSE: {train_rmse:.4f}")

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(12, 4))


plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layoimport numpy as npt()
plt.show()

In [ ]:




test_predictions = np.clip(test_predictions, 0, 5)
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'label': test_predictions.flatten()
})
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
print("Submission file created: /kaggle/working/submission.csv")

In [ ]:
print(pd.read_csv('/kaggle/working/submission.csv').columns.tolist())
print(pd.read_csv('/kaggle/working/submission.csv').shape)

In [ ]:

import os
import pandas as pd
from IPython.display import FileLink

submission_path = '/kaggle/working/submission.csv'


if os.path.exists(submission_path):
    print(f"Submission file exists at {submission_path}")
    submission_df = pd.read_csv(submission_path)
    print("Columns:", submission_df.columns.tolist())
    print("Shape:", submission_df.shape)
    print("First few rows:\n", submission_df.head())
    
  
    print("\nMain FileLink (check /kaggle/working/submission.csv):")
    display(FileLink(submission_path))
    
   
    alt_submission_path = '/kaggle/working/output/submission.csv'
    os.makedirs('/kaggle/working/output', exist_ok=True)
    submission_df.to_csv(alt_submission_path, index=False)
    print(f"Alternative submission file created: {alt_submission_path}")
    print(f"Alternative file size: {os.path.getsize(alt_submission_path)} bytes")
    print("\nAlternative FileLink (check /kaggle/working/output/submission.csv):")
    display(FileLink(alt_submission_path))
else:
    print(f"Submission file not found at {submission_path}")
    print("Files in /kaggle/working/:")
    print(os.listdir('/kaggle/working/'))


print("\nListing files in /kaggle/working/:")
print(os.listdir('/kaggle/working/'))
if os.path.exists('/kaggle/working/output'):
    print("Listing files in /kaggle/working/output/:")
    print(os.listdir('/kaggle/working/output'))


try:
    !mkdir -p ~/.kaggle
   
    !cp /kaggle/input/kaggle-json/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !mkdir -p /kaggle/working/api_output
    !kaggle kernels output shivanimehra06/notebookd97e4780df -p /kaggle/working/api_output/
    print("Files in /kaggle/working/api_output/:")
    print(os.listdir('/kaggle/working/api_output/'))
    api_submission_path = '/kaggle/working/api_output/submission.csv'
    if os.path.exists(api_submission_path):
        print(f"API-downloaded file size: {os.path.getsize(api_submission_path)} bytes")
        print(pd.read_csv(api_submission_path).head())
        display(FileLink(api_submission_path))
except Exception as e:
    print(f"Error using Kaggle API: {e}")
    print("Ensure kaggle.json is uploaded to /kaggle/input/kaggle-json/ and kernel slug is correct.")